In [1]:
# --- Imports ---
import requests
import datetime
import pandas as pd
import numpy as np
from meteostat import Point, Monthly
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim

In [2]:
# Tomorrow.io API Key
API_KEY = "mTgxFT8btimLVUoCY3ZkKQiwPxP6k0d6"

In [3]:

# --- Get latitude & longitude from location ---
def get_lat_lon(state, district, country="India"):
    try:
        geolocator = Nominatim(user_agent="weather_forecast_app")
        query = f"{district}, {state}, {country}"
        location = geolocator.geocode(query)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"⚠️ Could not find {query}, defaulting to Delhi.")
            return 28.7041, 77.1025
    except:
        print("⚠️ Error fetching location. Using default: Delhi")
        return 28.7041, 77.1025


# --- Fetch forecast from Tomorrow.io API ---
def get_forecast_tomorrowio(lat, lon, start_date, end_date):
    url = "https://api.tomorrow.io/v4/timelines"
    headers = {"apikey": API_KEY}
    params = {
        "location": f"{lat},{lon}",
        "fields": ["temperature", "precipitationIntensity", "windSpeed"],
        "startTime": start_date.isoformat(),
        "endTime": end_date.isoformat(),
        "timesteps": "1d"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None


# --- Predict weather from historical data (Meteostat) ---
def predict_weather_historical(lat, lon, month, year):
    try:
        location = Point(lat, lon)

        # Fetch historical monthly data (last 10 years)
        start = datetime.datetime(year - 10, month, 1)
        end = datetime.datetime(year - 1, month, 28)

        data = Monthly(location, start, end)
        df = data.fetch()

        if df.empty:
            return None

        # Drop rows with missing values (important fix ✅)
        df = df[['tavg', 'prcp']].dropna()

        if df.empty or len(df) < 3:   # too little data
            print("⚠️ Not enough historical data available.")
            return None

        # Train regression models
        X = np.array(df.index.year).reshape(-1, 1)
        y_temp = df['tavg'].values
        y_precip = df['prcp'].values

        model_temp = LinearRegression().fit(X, y_temp)
        model_precip = LinearRegression().fit(X, y_precip)

        # Predict for given year
        X_pred = np.array([[year]])
        predicted_temp = model_temp.predict(X_pred)[0]
        predicted_precip = model_precip.predict(X_pred)[0]

        return {
            "predicted_temperature": round(predicted_temp, 2),
            "predicted_precipitation_mm": round(predicted_precip, 2)
        }

    except Exception as e:
        print("⚠️ Error in historical prediction:", e)
        return None


# --- Interactive user inputs ---
state = input("Enter state name: ")
district = input("Enter district/city name: ")
month = int(input("Enter month (1-12): "))
year = int(input("Enter year (e.g., 2025): "))

# Get coordinates
lat, lon = get_lat_lon(state, district)

# Define forecast period
start_date = datetime.datetime(year, month, 1)
if month == 12:
    end_date = datetime.datetime(year + 1, 1, 1)
else:
    end_date = datetime.datetime(year, month + 1, 1)

# --- Forecast or fallback ---
forecast = get_forecast_tomorrowio(lat, lon, start_date, end_date)

if forecast and "data" in forecast and forecast["data"]["timelines"]:
    print("\n☁️ Weather Forecast from Tomorrow.io:")
    for interval in forecast["data"]["timelines"][0]["intervals"]:
        print(interval["startTime"],
              "| 🌡 Temp:", interval["values"]["temperature"], "°C",
              "| ☔ Precip:", interval["values"]["precipitationIntensity"], "mm/hr",
              "| 🌬 Wind:", interval["values"]["windSpeed"], "m/s")
else:
    print("\n⚠️ Tomorrow.io API unavailable. Using historical prediction...")
    prediction = predict_weather_historical(lat, lon, month, year)
    if prediction:
        print("📊 Predicted Weather (Historical Data):")
        print(f"🌡 Avg Temperature: {prediction['predicted_temperature']} °C")
        print(f"☔ Precipitation: {prediction['predicted_precipitation_mm']} mm")
    else:
        print("❌ No historical data available for this region.")



⚠️ Tomorrow.io API unavailable. Using historical prediction...
📊 Predicted Weather (Historical Data):
🌡 Avg Temperature: 25.89 °C
☔ Precipitation: 98.71 mm


In [6]:
import pickle

# Save trained weather model
filename = "weather_model.pkl"
with open(filename, "wb") as file:
    pickle.dump(Weather_Prediction_Final, file)

print(f"✅ Weather model saved as {filename}")


NameError: name 'Weather_Prediction_Final' is not defined